In [2]:
from pymatgen import Lattice, Structure, Molecule, Composition
from pymatgen.io.vasp import Poscar
import numpy as np
import pandas as pd
import random

In [5]:
supercell_dim = [5, 5, 1]
infile = "WSe2.vasp"

def get_chalcogen_sites(wse2,index):
    q = wse2.structure.get_all_neighbors(2.6,include_index=True)[index]
    chalc_choice = random.randint(0,5)
    reference = q[chalc_choice][0].x
    outlist = [q[chalc_choice][2]]
    for i in range(1,len(q)):
        if i != chalc_choice:
            if q[i][0].x == reference:
                outlist.append(q[i][2])
    if len(outlist) == 2:
        return sorted(outlist)
    else:
        return "error"
    
def get_unique_chalc_pairs(wse2, num_defects):
    num_W = int(wse2.structure.composition.as_dict()['W'])
    num_Se = int(wse2.structure.composition.as_dict()['Se'])
    if num_defects > num_Se/2:
        print("ERROR")
        return "Error"
    indexlist = []
    i = 0
    while i < num_defects:
        random_site = random.randint(0,num_W-1)
        chalc_inds = get_chalcogen_sites(wse2,random_site)
        if (chalc_inds not in indexlist) and (chalc_inds != "error"):
            indexlist.append(chalc_inds)
            i += 1
    return indexlist

def get_alloyed_structure(wse2,num_defects):
    chalc_list = get_unique_chalc_pairs(wse2, num_defects)

    for i in chalc_list:
        wse2.structure[int(i[0])] = "S"
        wse2.structure[int(i[1])] = "S"
    return Poscar(wse2.structure.get_sorted_structure())

wse2 = Poscar.from_file(infile)
wse2.structure.make_supercell(supercell_dim)
wse2 = Poscar(wse2.structure.get_sorted_structure())
num_W = int(wse2.structure.composition.as_dict()['W'])

for i in range(num_W):
    wse2 = Poscar.from_file(infile)
    wse2.structure.make_supercell(supercell_dim)
    wse2 = Poscar(wse2.structure.get_sorted_structure())
    
    wse2_new = get_alloyed_structure(wse2, num_defects=i)

    num_Se = int(wse2_new.structure.composition.as_dict()['Se'])
    num_S = int(wse2_new.structure.composition.as_dict()['S'])
    defect_concentration = round(num_S/(num_S + num_Se),2)

    print(defect_concentration, num_S, num_Se)

    wse2_new.write_file("WSe2_" + str(defect_concentration) + ".vasp")

0.0 0 50
0.04 2 48
0.08 4 46
0.12 6 44
0.16 8 42
0.2 10 40
0.24 12 38
0.28 14 36
0.32 16 34
0.36 18 32
0.4 20 30
0.44 22 28
0.48 24 26
0.52 26 24
0.56 28 22
0.6 30 20
0.64 32 18
0.68 34 16
0.72 36 14
0.76 38 12
0.8 40 10
0.84 42 8
0.88 44 6
0.92 46 4
0.96 48 2
